# XHARPy Example: Urea with Quantum Espresso
 - Program: Quantum Espresso
 - B3LYP calculation from PBE files
 - Core scaling

In [1]:
from xharpy import (shelxl_hkl2pd, cif2data, lst2constraint_dict, create_construction_instructions,
                    refine, write_cif, write_res, write_fcf, add_density_entries_from_fcf)
import os

In [2]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)

### Loading Data

In [3]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)

In [5]:
hkl = shelxl_hkl2pd('iam.hkl')

In [6]:
constraint_dict = lst2constraint_dict('iam.lst')

### Setting Options

In [7]:
refinement_dict = {
    'f0j_source': 'qe',
    'reload_step': 1,
    'core': 'scale',
}

In [8]:
computation_dict = {
    'symm_equiv': 'once',
    'mpicores': 2,
    'control': {
        'prefix': 'A23',
        'pseudo_dir': './pseudo/',
    },
    'system': {
        'ibrav': 6,
        'a': float(cell[0]),
        'c': float(cell[2]),
        'ecutwfc': 50,
        'ecutrho': 200,
        'input_dft': 'b3lyp'
    },
    'paw_files': {
        'H': 'H.pbe-kjpaw_psl.1.0.0.UPF',
        'C': 'C.pbe-n-kjpaw_psl.1.0.0.UPF',
        'O': 'O.pbe-n-kjpaw_psl.1.0.0.UPF',
        'N': 'N.pbe-n-kjpaw_psl.1.0.0.UPF'
    },
    'k_points':{
        'mode': 'automatic',
        'input': '1 1 1 0 0 0'
    }
}

### Refinement

In [9]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)


In [10]:
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-09 20:10:01.292875
Preparing
  calculating core density for H from H.pbe-kjpaw_psl.1.0.0.UPF
  calculating core density for C from C.pbe-n-kjpaw_psl.1.0.0.UPF
  calculating core density for O from O.pbe-n-kjpaw_psl.1.0.0.UPF
  calculating core density for N from N.pbe-n-kjpaw_psl.1.0.0.UPF
  calculating first atomic form factors
  calculated density, continuing with partitioning
  building least squares function
  setting up gradients
step 0: Optimizing scaling
  wR2: 0.053475, number of iterations: 1
  minimizing least squares sum
  wR2: 0.021016, number of iterations: 45
step 1: calculating new structure factors
  calculated density, continuing with partitioning
  minimizing least squares sum
  wR2: 0.017450, number of iterations: 39
step 2: calculating new structure factors
  calculated density, continuing with partitioning
  minimizing least squares sum
  wR2: 0.017626, number of iterations: 34
step 3: calculating new structure factors
  calculated de

### Writing Data to Disk

In [11]:
write_cif(
    output_cif_path=os.path.join(output_folder, 'xharpy.cif'),
    cif_dataset='xharpy',
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)

In [12]:
write_fcf(
    fcf_path=os.path.join(output_folder, 'xharpy.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [13]:
write_fcf(
    fcf_path=os.path.join(output_folder, 'xharpy_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [14]:
write_res(
    out_res_path=os.path.join(output_folder, 'xharpy_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [15]:
add_density_entries_from_fcf(
    os.path.join(output_folder, 'xharpy.cif'),
    os.path.join(output_folder, 'xharpy_6.fcf')
)